In [1]:
import numpy as np

In [8]:
word2vec = {}
with open("../word2vec/W2V_150.txt", mode='r', encoding='utf-8') as file:
    next(file)
    next(file)
    lines = file.readlines()
    for line in lines:
        split = line.split()

        word = split[0]
        vec = split[1:]

        word2vec[word] = np.array(vec).astype(float)


In [11]:
interesting = word2vec["thú_vị"]
print(len(interesting))
print(interesting.dtype)

150
float64


In [12]:
fun = word2vec["vui"]

In [15]:
np.dot(fun.T, interesting)

82.74809785022461

# Problem 1:

In [19]:
def cosine_similarity(vec1: np.array, vec2: np.array) -> float:
    numerator = np.sum(np.multiply(vec1, vec2))
    denominator = np.sqrt(np.dot(vec1.T, vec1)) * np.sqrt(np.dot(vec2.T, vec2))

    return numerator / denominator

In [20]:
cosine_similarity(fun, interesting)

0.48814743997004356

In [32]:
def KNN(word: str, vocab: dict, k: int = 5) -> np.array:
    distance_list = []
    for w in vocab:
        if w != word:
            distance_list.append((w, cosine_similarity(vocab[word], vocab[w])))
    distance_list = sorted(distance_list, key=lambda x: x[1], reverse=True)

    return np.array(distance_list[:k])

In [33]:
KNN("vả", word2vec)

array([['tát', '0.5022767817350094'],
       ['đấm', '0.4383916557515601'],
       ['bạt_tai', '0.4378275347842416'],
       ['đánh_liên_tiếp', '0.4205850772757408'],
       ['đánh_túi_bụi', '0.41529420942501954']], dtype='<U32')